In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

1.
Read in the CSV file with the information on challenges, search URLs, and number of pages per challenge
from the https://eunitedvsvirus.devpost.com/submissions page

2.
For each row in the CSV file:
* get the Challenge and the Sub-challenge
* extract the search URL, to be added to https://eunitedvsvirus.devpost.com/submissions
* For 1:NumPages, get the project URLs from https://eunitedvsvirus.devpost.com/submissions + search URL
* Create a new data frame, where each row has: project URL; Challenge; Sub-challenge

3.
Save the new data frame to a CSV file ProjectURLs.csv

In [53]:
# Define the EUvsVirus devpost URL

euhack_url = 'https://eunitedvsvirus.devpost.com/'

In [44]:
# Load the CSV with the Challenge, SubChallenge, and search URLs

infoDF = pd.read_csv("Data/WebPageInfo.csv")
infoDF.head()

,Challenge,SubChallenge,NumPages,SearchText
0,Health,Equipment,6,search?utf8=✓&terms=&sort=recent&filter[challe...
1,Health,Ventilators,3,search?utf8=✓&terms=&sort=recent&filter[challe...
2,Health,MedPersonnel,3,search?utf8=✓&terms=&sort=recent&filter[challe...
3,Health,CommunicationPrevention,13,search?utf8=✓&terms=&sort=recent&filter[challe...
4,Health,RapidTests,3,search?utf8=✓&terms=&sort=recent&filter[challe...


In [61]:
def get_ProjURLs(searchText, pNum):
    '''
    Given the search URL, and the page number, scrape the URLs of the
    projects corresponding to the search URL on that page.
    The search criteria correspond to Challenge and SubChallenge that a project was 
    submitted to.
    
    returns all the project URLs that correspond to the given search criteria on the given page number
    '''
    
    searchURL = searchText + "&page=" + str(pNum)
    req = requests.get(f'{euhack_url}submissions/{searchURL}')
    Projsoup = BeautifulSoup(req.text, 'html.parser')
    ProjHREF = Projsoup.find_all(href = re.compile("https://devpost.com/software/"))
    ProjURL = [l.get("href") for l in ProjHREF]
    return ProjURL

In [60]:
def get_AllProjURLs(searchObj):
    '''
    A searchObj corresponds to one row in the infoDF, and contains the following information:
    * the SearchText, which are the search criteria to determine the project;
    * the number of pages returned by the search;
    * the Challenge that the project was submitted for;
    * the SubChallenge that the project was submitted for
    Given the search URL, and the number of pages returned by a given search,
    repeat the search request number of pages times, to scrape the URLs of the
    projects corresponding to the search criteria.
    
    returns all the project URLs
    '''
    searchURL = searchObj.SearchText
    nPages = searchObj.NumPages + 1
    Chal = searchObj.Challenge
    subChal = searchObj.SubChallenge
    print(searchURL)
    
    allProjURL = [get_ProjURLs(searchURL, i) for i in range(1, nPages)]
    allProjURL = list(pd.core.common.flatten(allProjURL))
    projURLdf = pd.DataFrame({'ProjURL':allProjURL, 'Challenge':Chal, 'SubChallenge':subChal})
    print("done")
    return projURLdf

In [50]:
# Iterate through all rows of the infoDF
# to collect the project URLs corresponding to projects submitted to a given Challenge & SubChallenge

dfList = list()
for r in infoDF.itertuples():
    newL = get_AllProjURLs(r)
    dfList.append(newL)
    
projURLdf = pd.concat(dfList, ignore_index = True)

search?utf8=✓&terms=&sort=recent&filter[challenge][]=1+|+health+%26+life+|+protective+equipment
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=2+|+health+%26+life+|+ventilators%2Frespirators
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=3+|+health+%26+life+|+protection+of+medical+personnel
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=4+|+health+%26+life+|+real+time+communication+%26+prevention
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=5+|+health+%26+life+|+cheap+rapid+tests
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=6+|+health+%26+life+|+lack+of+skilled+caregivers
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=7+|+health+%26+life+|+research
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=8+|+health+%26+life+|+other
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=9+|+business+continuity+|+efficient+team+work
done
search?utf8=✓&terms=&sort=recent&filter[challenge][]=10+|+business+continuity+

In [52]:
# save the results to CSV file

projURLdf.to_csv("Data/projectURLs.csv")